In [7]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm
import os 

In [6]:
#Prefilter out all npidata csv columns that we dont need, created a new csv with the column names to keep
save_cols = pd.read_csv('./NPPES_Data_Dissemination_February_2023/new_columns.csv', header=None)
save_cols_list = save_cols.iloc[0,:].tolist()

In [5]:
def create_database(save_cols):

    #Create the 5 tables from DocGraph and NPPES downloads, 
    # and the 2 tables from the NUCC taxonomy and Zip to CBSA info 
    # and add them to a new data base called 'hop_db.sqlite'
    
    db = sqlite3.connect('./data/hop_db.sqlite')

    for chunk in tqdm(pd.read_csv('./DocGraph_Hop_Teaming_2018_Commercial/DocGraph_Hop_Teaming_2018.csv', chunksize=100000)):
        chunk = chunk[(chunk['transaction_count'] >= 50) & (chunk['average_day_wait'] < 50)] #Prefiltering accidental referrals
        chunk.to_sql('hop', db, if_exists = 'append', index = False) 

    for chunk in tqdm(pd.read_csv('./NPPES_Data_Dissemination_February_2023/endpoint_pfile_20050523-20230212.csv', chunksize=10000)):
        chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
        chunk.to_sql('endpoint', db, if_exists = 'append', index = False) 

    for chunk in tqdm(pd.read_csv('./NPPES_Data_Dissemination_February_2023/npidata_pfile_20050523-20230212.csv', dtype=str, chunksize=100000, usecols=save_cols_list)):
        chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
        chunk.to_sql('npidata', db, if_exists = 'append', index = False)

    for chunk in tqdm(pd.read_csv('./NPPES_Data_Dissemination_February_2023/othername_pfile_20050523-20230212.csv', chunksize=10000)):
        chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
        chunk.to_sql('othername', db, if_exists = 'append', index = False)     

    for chunk in tqdm(pd.read_csv('./NPPES_Data_Dissemination_February_2023/pl_pfile_20050523-20230212.csv', chunksize=10000)):
        chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
        chunk.to_sql('pl', db, if_exists = 'append', index = False)
    
    # 2 auxilary tables from other  
    for chunk in tqdm(pd.read_csv('./data/nucc_taxonomy_230.csv', encoding='unicode_escape', chunksize=1000)):
        chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
        chunk.to_sql('nucc_taxonomy', db, if_exists = 'append', index = False)  

    for chunk in tqdm(pd.read_csv('./data/ZIP_CBSA.csv', chunksize=10000)):
        chunk.to_sql('zip_to_cbsa', db, if_exists = 'append', index = False)  

    db.close()

In [8]:

if os.path.isfile('./data/hop_db.sqlite') != True: # Make sure the data base isn't already created, this is a large DB
    create_database(save_cols = save_cols_list)


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]